In [ ]:
!pip install langchain huggingface_hub openai transformers

In [ ]:
!pip install langchain_community
from langchain import PromptTemplate , HuggingFaceHub , LLMChain

In [ ]:
pip install transformers

In [ ]:
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification, AutoModelForSeq2SeqLM,AutoModelForSequenceClassification
import tensorflow as tf
import numpy as np

In [ ]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
text = "i love all the feature provided by AI"

In [ ]:
input = tokenizer(text , return_tensors='tf')

In [ ]:
output = model(**input)

In [ ]:
prob = tf.nn.softmax(output.logits,axis=-1).numpy()[0]

In [ ]:
predict_class_id = np.argmax(prob)
model.config.id2label[predict_class_id]

Language translation model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

In [ ]:
input_text  = "my name is dev , i use to learn data science"
tokenizer.src_lang = "eng_Latn"
encoded  = tokenizer(input_text,return_tensors='pt')

In [ ]:
output = model.generate(**encoded,forced_bos_token_id = tokenizer.convert_tokens_to_ids("hin_Deva"))

In [ ]:
output_text = tokenizer.batch_decode(output,skip_special_tokens=True)

In [ ]:
output_text

In [ ]:
pip install datasets

Fine tuning a Model on our own dataset so we are fine tuning a dataset on imdb dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset('imdb')

In [ ]:
dataset

In [ ]:
dataset["train"]

In [ ]:
dataset["train"][0]

In [ ]:
from transformers import TrainingArguments , Trainer

In [ ]:
model_checkpoint = "distilbert-base-uncased"
tokenizer= AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def tokenizer_func(data):
  return tokenizer(data['text'],padding=True,truncation=True)

In [ ]:
tokenized_dataset=dataset.map(tokenizer_func,batched=True)

In [ ]:
tokenized_dataset

Model will not be able to understand text also we have got input_ids for it input_labels we can use and attention_mak also attention_mask is of length 512 as no sentence can be greater than 512 length and labels are in numerical so we can include them in training

In [ ]:
len(tokenized_dataset["train"][0]["attention_mask"])

As you can consider below first sentence is containing 288 words and second sentence is containing 214

In [ ]:
len(tokenized_dataset["train"][0]["text"].split())

In [ ]:
len(tokenized_dataset["train"][1]["text"].split())

In [ ]:
tokenized_dataset= tokenized_dataset.remove_columns(["text"])

In [ ]:
tokenized_dataset

In [ ]:
tokenized_dataset.set_format("torch")

In [ ]:
tokenized_dataset["train"].column_names

In [ ]:
tokenized_dataset= tokenized_dataset.rename_column("label","labels")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels = 2 )

In [ ]:
from transformers import TrainingArguments , Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir="./result",
    eval_strategy="epoch",
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=.01
)

In [ ]:
from google.colab import userdata

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')

In [ ]:
trainer = Trainer(model=model,
        args = training_args,
        train_dataset=tokenized_dataset['train'].select(range(100)),
        eval_dataset=tokenized_dataset['test'].select(range(100))
        )

trainer.train()